Natural Language Processing

1. Split feature and target
2. Tokenization
3. Bag of Words (BoW)
4. n grams
5. tfidf

6. PREDICT

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CDSS/Data Folder/Amazon_Unlocked_Mobile.csv', encoding ="utf8")
df = df.sample(frac=1, random_state=10) #to create a random environment

Mounted at /content/drive


In [3]:
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
394349,Sony XPERIA Z2 D6503 FACTORY UNLOCKED Internat...,NaN,244.95,5,Very good one! Better than Samsung S and iphon...,0.0
34377,Apple iPhone 5c 8GB (Pink) - Verizon Wireless,Apple,194.99,1,"The phone needed a SIM card, would have been n...",1.0
248521,Motorola Droid RAZR MAXX XT912 M Verizon Smart...,Motorola,174.99,5,I was 3 months away from my upgrade and my Str...,3.0
167661,CNPGD [U.S. Office Extended Warranty] Smartwat...,CNPGD,49.99,1,an experience i want to forget,0.0
73287,Apple iPhone 7 Unlocked Phone 256 GB - US Vers...,Apple,922.00,5,GREAT PHONE WORK ACCORDING MY EXPECTATIONS.,1.0


In [4]:
len(df)

413840

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 413840 entries, 394349 to 345353
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Product Name  413840 non-null  object 
 1   Brand Name    348669 non-null  object 
 2   Price         407907 non-null  float64
 3   Rating        413840 non-null  int64  
 4   Reviews       413770 non-null  object 
 5   Review Votes  401544 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 22.1+ MB


In [11]:
df.isnull().sum()

Product Name        0
Brand Name          0
Price               0
Rating              0
Reviews             0
Review Votes        0
Positively Rated    0
dtype: int64

In [7]:
df.Rating.unique()

array([5, 1, 4, 2, 3])

In [8]:
# Drop missing values
df.dropna(inplace=True)

# Remove any 'neutral' ratings equal to 3
df = df[df['Rating'] != 3]

# Encode 4s and 5s as 1 (rated positively)
# Encode 1s and 2s as 0 (rated poorly)
# if greater than 3, make it as 1, otherwise 0
df['Positively Rated'] = np.where(df['Rating'] > 3, 1, 0)

In [9]:
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Positively Rated
34377,Apple iPhone 5c 8GB (Pink) - Verizon Wireless,Apple,194.99,1,"The phone needed a SIM card, would have been n...",1.0,0
248521,Motorola Droid RAZR MAXX XT912 M Verizon Smart...,Motorola,174.99,5,I was 3 months away from my upgrade and my Str...,3.0,1
167661,CNPGD [U.S. Office Extended Warranty] Smartwat...,CNPGD,49.99,1,an experience i want to forget,0.0,0
73287,Apple iPhone 7 Unlocked Phone 256 GB - US Vers...,Apple,922.00,5,GREAT PHONE WORK ACCORDING MY EXPECTATIONS.,1.0,1
277158,Nokia N8 Unlocked GSM Touch Screen Phone Featu...,Nokia,95.00,5,I fell in love with this phone because it did ...,0.0,1


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 308270 entries, 34377 to 345353
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Product Name      308270 non-null  object 
 1   Brand Name        308270 non-null  object 
 2   Price             308270 non-null  float64
 3   Rating            308270 non-null  int64  
 4   Reviews           308270 non-null  object 
 5   Review Votes      308270 non-null  float64
 6   Positively Rated  308270 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 18.8+ MB


# Model Selection in scikit-learn

In [12]:
from sklearn.model_selection import train_test_split

# Split data into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(df['Reviews'],
                                                    df['Positively Rated'],
                                                    random_state=0)

In [13]:
print( X_train.iloc[2])



Had the phone for less than a week and it died. No warning. Screen is great. 3g on ATT is fast.Doesn't matter about the features though. Dying in a few days is a sure way to get 1 star.Returned for refund. I would give it a star for that, but that is Amazon's star.


In [14]:
X_train.shape

(231202,)

In [15]:
X_test.shape

(77068,)

# Extracting features from text files


Text files are actually series of words (ordered). In order to run machine learning algorithms we need to convert the text files into numerical feature vectors. We will be using bag of words model.

## Bag-of-words (BOW)
BOW model allows us to represent text as numerical feature vectors. The idea behind BOW is quite simple and can be summarized as follows:
- 1) Create a vocabulary of unique tokens (or words) from the entire set
    of documents.
- 2) Construct a feature vector from each document that contains the counts of how often each word occurs in the particular document.

Since the unique words in each document represent only a small subset of all the words in the bag-of-words vocabulary, the feature vectors will consist of mostly zeros, which is why we call them sparse. For this reason we say that bags of words are typically <b>high-dimensional sparse datasets</b>.

{for our example. Briefly, we segment each text file into words (for English splitting by space), and count # of times each word occurs in each document and finally assign each word an integer id. Each unique word in our dictionary will correspond to a feature (descriptive feature).}


### Transform words into vectors (CountVectorizer)
To construct a bag-of-words model based on the word counts in the respective documents, we can use the `CountVectorizer` class implemented in `scikit-learn`. As we will see in the following codes, the `CountVectorizer` class takes an array of text data, which can be documents or just sentences, and constructs the bag-of-words model for us:

Scikit-learn has a high level component which will create feature vectors for us <b>‘CountVectorizer’</b>

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining and the weather is sweet'])

# Fit the CountVectorizer to the training data
vect1=CountVectorizer().fit(docs)

# transform the documents in the training data to a document-term matrix.
bag1 = vect1.transform(docs)

#vector is number format

In [17]:
vect1.vocabulary_

{'the': 5, 'sun': 3, 'is': 1, 'shining': 2, 'weather': 6, 'sweet': 4, 'and': 0}

In [18]:
vect1.get_feature_names_out()

array(['and', 'is', 'shining', 'sun', 'sweet', 'the', 'weather'],
      dtype=object)

## <font color=green> Exercise A</font>

1) Do CountVectorizer for training data

2) Determine:
- The number of features
- The shape of sparse matrix

In [55]:
from sklearn.feature_extraction.text import CountVectorizer

# Fit the CountVectorizer to the training data
review1 = CountVectorizer().fit(X_train)

# transform the documents in the training data to a document-term matrix.
X_train_vectorized1 = review1.transform(X_train)


print(len(review1.get_feature_names_out()))
X_train_vectorized1.shape

# for BOW, raw data is review1, and bag is vectorized1
# output
# (num of observation, num of columns/keywords)

53490


(231202, 53490)

In [21]:
X_test_vectorized1 = review1.transform(X_test)
X_test_vectorized1.shape

(77068, 53490)

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Split data into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(df['Reviews'],
                                                    df['Positively Rated'],
                                                    random_state=0)

# Train the Logistic Regression model
lm = LogisticRegression()
lm.fit(X_train_vectorized1, y_train)

# Evaluate the model on the test data
accuracy = lm.score(X_test_vectorized1, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9440494109098458


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
review1.vocabulary_

{'so': 43810,
 'far': 19305,
 'good': 21913,
 'ordered': 33476,
 'it': 26161,
 'feb': 19538,
 '2015': 937,
 'rcvd': 38458,
 '10': 133,
 'no': 32150,
 'problem': 36895,
 'getting': 21557,
 'sim': 42954,
 'card': 9782,
 'having': 23154,
 'the': 47090,
 'phone': 35127,
 'activated': 3688,
 'really': 38579,
 'signs': 42916,
 'of': 32844,
 'used': 50052,
 'that': 47058,
 'can': 9609,
 'see': 41956,
 'great': 22296,
 'seller': 42062,
 'beautiful': 7337,
 'well': 51686,
 'worth': 52534,
 'asking': 5921,
 'price': 36707,
 'and': 4972,
 'more': 30808,
 'had': 22752,
 'for': 20304,
 'less': 27769,
 'than': 46991,
 'week': 51619,
 'died': 15131,
 'warning': 51328,
 'screen': 41691,
 'is': 26055,
 '3g': 1704,
 'on': 33096,
 'att': 6141,
 'fast': 19364,
 'doesn': 15978,
 'matter': 29450,
 'about': 3282,
 'features': 19512,
 'though': 47400,
 'dying': 16572,
 'in': 24757,
 'few': 19641,
 'days': 13868,
 'sure': 45856,
 'way': 51493,
 'to': 47795,
 'get': 21534,
 'star': 44821,
 'returned': 40312,
 '

In [24]:
review1.get_feature_names_out()

array(['00', '000', '0000', ..., '这不是欧美版的', '非常赞', '좋군'], dtype=object)

# Logistic Regression classification

We will train a logistic regression model to classify the  Amazon reviews into positive and negative reviews by using feature matrix.

In [25]:
from sklearn.metrics import roc_auc_score

# Predict the transformed test documents
predictions = lm.predict(review1.transform(X_test))
y_proba = lm.predict_proba(review1.transform(X_test))

print('AUC: ', roc_auc_score(y_test, y_proba[:,1]))

AUC:  0.9755408245377601


In [ ]:
# coefficient of each keywords
# lower coefficient = -ve review
lm.coef_

array([[-0.92874463, -0.17081246,  0.01935562, ...,  0.10821892,
         0.08218676,  0.08218676]])

In [26]:
lm.coef_[0].argsort()

array([52527, 26745, 49755, ..., 18567, 18392, 18393])

In [27]:
# get the feature names as numpy array
feature_names = np.array(review1.get_feature_names_out())

# Sort the coefficients from the model
sorted_coef_index = lm.coef_[0].argsort()

# Find the 10 smallest and 10 largest coefficients
print('Smallest Coefs:' )
print(feature_names[sorted_coef_index[  :10]])

print('\n Largest Coefs:')   #\n is for line space
print(feature_names[sorted_coef_index[  :-11:-1]]) #select last 10 values

Smallest Coefs:
['worst' 'junk' 'unusable' 'garbage' 'useless' 'disappointing' 'horrible'
 'luck' 'shattered' 'terrible']

 Largest Coefs:
['excelente' 'excelent' 'exelente' 'loves' 'loving' 'excellent' 'perfecto'
 'complaints' 'love' 'bien']


# Tfidf

When we are analyzing text data, we often encounter words that occur across multiple documents from both classes. Those frequently occurring words typically don't contain useful or discriminatory information. In this subsection, we will learn about a useful technique called **term frequency-inverse document frequency** (*tf-idf*) that can be used to downweight those frequently occurring words in the feature vectors. On the other words by tf-idf we can reduce the weightage of more common words like (the, is, an etc.) which occurs in all document.

The *tf-idf* can be defined as the product of the term frequency and the inverse document frequency:

\begin{align}
\textit{tf-idf}(t,d) = tf(t,d) \times idf(t,d)
\end{align}

Here the <font color=green><b> *tf(t,d)* </b></font> is the term frequency that equal to **Count of word / Total words, in each document**. The inverse document frequency *idf(t,d)* can be calculated as:

\begin{align}
idf(t,d) = log\frac{n_d}{\text{df(d,t)}}
\end{align}

where <font color=green><b> $n_d$ </b></font> is **the total number of documents**, and <font color=green><b>*df(d,t)*</b></font> is **the number of documents *d* that contain the term t**. Note that the log is used to ensure that low document frequencies are not given too much weight.


scikit-learn implements yet another vectorizer, the TfidfVectorizer, that creates feature vectors as tf-idfs.


In [30]:
# to remove standard english stop words
from sklearn.feature_extraction.text import TfidfVectorizer

docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining and the weather is sweet'])

vect2 = TfidfVectorizer().fit(docs)
bag2 = vect2.transform(docs)

# trained data is vect2
# bag data is bag2

In [31]:
vect2.get_feature_names_out()

array(['and', 'is', 'shining', 'sun', 'sweet', 'the', 'weather'],
      dtype=object)

In [35]:
vect3 = TfidfVectorizer(min_df=2).fit(docs)
bag3 = vect3.transform(docs)
vect3.get_feature_names_out()

array(['is', 'shining', 'sun', 'sweet', 'the', 'weather'], dtype=object)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Fit the TfidfVectorizer to the training data
review2 = TfidfVectorizer(min_df=5).fit(X_train)
X_train_vectorized2 = review2.fit_transform(X_train)

In [37]:
len(review2.get_feature_names_out())

17989

In [38]:
X_test_vectorized2 = review2.transform(X_test)
X_test_vectorized2.shape

(77068, 17989)

In [39]:
# Train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train_vectorized2, y_train)

# Evaluate the model on the test data
accuracy = model.score(X_test_vectorized2, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9493564125188145


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
X_train_vectorized2 = review2.transform(X_train)

model = LogisticRegression()
model.fit(X_train_vectorized2, y_train)

predictions = model.predict(review2.transform(X_test))
y_proba = model.predict_proba(review2.transform(X_test))

print('AUC: ', roc_auc_score(y_test, y_proba[:,1]))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AUC:  0.98255131869347


## <font color=green> Exercise B</font>
- Predict two below reviews as negative or positive using our model:

      ['no an issue, phone is working', 'an issue, phone is not working']       

In [40]:
model.predict(review2.transform(['amazing, phone is working', 'an issue, phone is not working']))

array([1, 0])

Try this!

In [41]:
model.predict(review2.transform(['no an issue, phone is working','an issue, phone is not working']))

array([0, 0])

# n-grams

The sequence of items in the bag-of-words model that we just created is also called the 1-gram or unigram model — each item or token in the vocabulary represents a single word. Generally, <b>the contiguous sequences of items in NLP</b> — words, letters, or symbols— is also called an n-gram. The choice of the number n in the n-gram model depends on the particular application. For instance, spam filtering applications tend to use n=3 or n=4 for good performances.
To summarize the concept of the n-gram representation, the 1-gram and 2-gram representations of our first document "the sun is shining" would be constructed as follows:
- 1-gram: "the", "sun", "is", "shining"
- 2-gram: "the sun", "sun is", "is shining"

The CountVectorizer class in scikit-learn allows us to use different n-gram models via its ngram_range parameter. By default, it uses a 1-gram representation.

In [42]:
# Try 2-gram representation
docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining and the weather is sweet'])

vect3=CountVectorizer(ngram_range=(1,2)).fit(docs)
bag3=vect3.transform(docs)

In [43]:
vect3.vocabulary_

{'the': 10,
 'sun': 7,
 'is': 2,
 'shining': 5,
 'the sun': 11,
 'sun is': 8,
 'is shining': 3,
 'weather': 13,
 'sweet': 9,
 'the weather': 12,
 'weather is': 14,
 'is sweet': 4,
 'and': 0,
 'shining and': 6,
 'and the': 1}

In [44]:
review3 = CountVectorizer(min_df=5, ngram_range=(1,2)).fit(X_train)

X_train_vectorized3 = review3.transform(X_train)

In [47]:
len(review3.get_feature_names_out())

199854

In [48]:
X_test_vectorized3 = review3.transform(X_test)
X_test_vectorized3.shape

(77068, 199854)

In [49]:
# Train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train_vectorized3, y_train)

# Evaluate the model on the test data
accuracy = model.score(X_test_vectorized3, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9748144495769969


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [50]:
model = LogisticRegression()
model.fit(X_train_vectorized3, y_train)

predictions = model.predict(review3.transform(X_test))
y_proba = model.predict_proba(review3.transform(X_test))

print('AUC: ', roc_auc_score(y_test, y_proba[:,1]))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AUC:  0.9897107306649673


In [52]:
feature_names = np.array(review3.get_feature_names_out())

sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:' )
print(feature_names[sorted_coef_index[:10]])

print('\n Largest Coefs:')
print(feature_names[sorted_coef_index[:-11:-1]])

Smallest Coefs:
['no good' 'not good' 'not worth' 'terrible' 'junk' 'worst'
 'not satisfied' 'horrible' 'not happy' 'nope']

 Largest Coefs:
['excelente' 'excelent' 'not bad' 'excellent' 'exelente' 'perfect'
 'no problems' 'awesome' 'perfecto' 'great']


In [54]:
sorted_coef_index

array([111971, 113416, 114180, ..., 113065,  57154,  57194])

In [53]:
# These reviews are now correctly identified
print(model.predict(review3.transform(['no an issue, phone is working',
                                    'an issue, phone is not working'])))

[1 0]
